In [1]:
%load_ext autoreload
%autoreload 2
import sys, os, socket
os.getcwd(), socket.gethostname()

('/home/ajasja/projects/truncator', 'dig60')

In [2]:
import numpy
import numpy as np

from glob import glob
import os

import truncator

import pymol
from pymol import cmd

In [3]:
pdb_files = glob("out/R3/01_cut/*/*.pdb");


In [4]:
pick_name = "out/R3/01_cut/4.5H/ZCON_127__numH4.5__from-20.77__to21.47_grC-ABD"

split_name = pick_name.split("__")
base_name = split_name[0]
group_name = split_name[-1].replace('gr','')
gsplit = group_name.split("-");
gsplit[0]=str(len(gsplit[0]))
gsplit[1]=str(len(gsplit[1]))
topology=gsplit[0]+gsplit[1]
#remove gr
struct_name=pick_name[:-9]
base_name,group_name,topology

#TODO figure out all connections acording to UP/DOWN status of helices
{'A-BCD':"[A,B+C+D] [A,D+C+B]".split(), 
 'AB-CD':"[A+B,C+D] [B+A,C+D] [A+B,D+C] [B+A,D+C] [C+D,A+B] [C+D,B+A] [D+C,A+B] [D+C,B+A]".split(), 
 'AD-BC':"[A+D,C+B] [D+A,C+B] [A+D,B+C] [D+A,B+C] [C+B,A+D] [C+B,D+A] [B+C,A+D] [B+C,D+A]".split(), 
 'B-ACD':"[B,A+C+D] [B,D+C+A]".split(), 
 'C-ABD':"[C,B+A+D] [C,D+A+B]".split(), 
 'D-ABC':"[D,A+B+C] [D,C+A+B]".split()};


In [5]:
struct_name=truncator.find_input(struct_name, pdb_files); struct_name

'out/R3/01_cut/4.5H/ZCON_127__numH4.5__from-20.77__to21.47.pdb'

In [7]:
%%time
reloop_result = truncator.reloop(struct_name, "out/R3/TEST03", 
                          script_name='truncator/xml/15_redesign_loops_tl_gen_profile_SymAnnVARS.xml', 
                          rosetta_bin="/home/brunette/src/Rosetta_dev/main/source/bin/rosetta_scripts.hdf5.linuxgccrelease",
                          extra_flags="-chemical:exclude_patches LowerDNA  UpperDNA Cterm_amidation VirtualBB ShoveBB VirtualDNAPhosphate VirtualNTerm CTermConnect sc_orbitals pro_hydroxylated_case1 pro_hydroxylated_case2 ser_phosphorylated thr_phosphorylated  tyr_phosphorylated tyr_sulfated lys_dimethylated lys_monomethylated  lys_trimethylated lys_acetylated glu_carboxylated cys_acetylated tyr_diiodinated N_acetylated C_methylamidated MethylatedProteinCterm",
                          structure_store="/home/brunette/DBs/hdf5/ss_grouped_vall_helix_shortLoop.h",
                          chain_connections='[A,D+C+B]',
                          loopLengthRange="2,5",
                          resAdjustmentRangeSide1="-3,1", 
                          resAdjustmentRangeSide2="-3,1",
                          RMSthreshold='.5',
                          test_run=False, verbose=True, add_suffix=True)

print(reloop_result.score_file)

/home/brunette/src/Rosetta_dev/main/source/bin/rosetta_scripts.hdf5.linuxgccrelease  \
-parser:protocol truncator/xml/15_redesign_loops_tl_gen_profile_SymAnnVARS.xml \
-s out/R3/01_cut/4.5H/ZCON_127__numH4.5__from-20.77__to21.47.pdb \
-indexed_structure_store:fragment_store  /home/brunette/DBs/hdf5/ss_grouped_vall_helix_shortLoop.h \
-out:path:pdb out/R3/TEST03 \
-out:suffix __ccA-DCB__llr2,5__ar1-3,1__ar2-3,1__rms.5 \
-out:file:scorefile  /home/ajasja/projects/truncator/out/R3/TEST03/ZCON_127__numH4.5__from-20.77__to21.47__ccA-DCB__llr2,5__ar1-3,1__ar2-3,1__rms.5.sc \
-beta \
-in:file:fullatom \
-renumber_pdb 1 \
-out:file:pdb_comments true \
-run:preserve_header true \
-out:file:scorefile_format json \
-out:pdb \
-parser:script_vars chain_connections="[A,D+C+B]" loopLengthRange="2,5" resAdjustmentRangeSide1="-3,1" resAdjustmentRangeSide2="-3,1" allowed_loop_abegos="AGBA,ABBA,AGBBA,ABABA,ABBBA,AGABBA,ABBBBA,AGBBBA" RMSthreshold=".5"  \
-out:no_nstruct_label \
-chemical:exclude_patches

In [6]:
import dask, distributed
client = distributed.Client(scheduler_file='scheduler.json'); client

Client Scheduler: tcp://172.16.131.90:7991 Dashboard: http://172.16.131.90:8787/status,Cluster Workers: 47 Cores: 47 Memory: 94.00 GB


In [7]:
pdb_files = glob("out/R3/01_cut/*/*.pdb");
regrouped_files = glob("out/R3/02_regruped/*/*.pdb"); 
picks = truncator.read_file_lines("out/R3/Reloop.picks", skip_comments=True)
for pick_name in picks:
    struct_name=pick_name[:-9]
    struct_name=truncator.find_input(struct_name, pdb_files)
    full_pick_name=truncator.find_input(pick_name, regrouped_files)
    print(f"load {full_pick_name}")
    print(f"load {struct_name}")
    print("")
        
   
  

load out/R3/02_regruped/4.5H/ZCON_37__numH4.5__from-21.21__to22.24__grAB-CD.pdb
load out/R3/01_cut/4.5H/ZCON_37__numH4.5__from-21.21__to22.24.pdb

load out/R3/02_regruped/4.5H/ZCON_37__numH4.5__from-21.21__to22.24__grA-BCD.pdb
load out/R3/01_cut/4.5H/ZCON_37__numH4.5__from-21.21__to22.24.pdb

load out/R3/02_regruped/4.5H/ZCON_15_re_loop__numH4.5__from-6.83__to32.93__grAD-BC.pdb
load out/R3/01_cut/4.5H/ZCON_15_re_loop__numH4.5__from-6.83__to32.93.pdb

load out/R3/02_regruped/4.5H/ZCON_15_re_loop__numH4.5__from-6.83__to32.93__grC-ABD.pdb
load out/R3/01_cut/4.5H/ZCON_15_re_loop__numH4.5__from-6.83__to32.93.pdb

load out/R3/02_regruped/4.5H/ZCON_127__numH4.5__from-20.77__to21.47__grAD-BC.pdb
load out/R3/01_cut/4.5H/ZCON_127__numH4.5__from-20.77__to21.47.pdb

load out/R3/02_regruped/4.5H/ZCON_127__numH4.5__from-20.77__to21.47__grC-ABD.pdb
load out/R3/01_cut/4.5H/ZCON_127__numH4.5__from-20.77__to21.47.pdb

load out/R3/02_regruped/4.5H/ZCON_13__numH4.5__from-17.52__to22.91__grAD-BC.pdb
load o

In [8]:
#TODO figure out all connections acording to UP/DOWN status of helices
gr_map= \
{'A-BCD':"[A,B+C+D] [A,D+C+B]".split(), 
 'AB-CD':"[A+B,C+D] [B+A,C+D] [A+B,D+C] [B+A,D+C] [C+D,A+B] [C+D,B+A] [D+C,A+B] [D+C,B+A]".split(), 
 'AD-BC':"[A+D,C+B] [D+A,C+B] [A+D,B+C] [D+A,B+C] [C+B,A+D] [C+B,D+A] [B+C,A+D] [B+C,D+A]".split(), 
 'B-ACD':"[B,A+C+D] [B,D+C+A]".split(), 
 'C-ABD':"[C,B+A+D] [C,D+A+B]".split(), 
 'D-ABC':"[D,A+B+C] [D,C+A+B]".split()};


In [ ]:
n = 0
res = []
#pick_name = "ZCON_37__numH4.5__from-21.21__to22.24__grAB-CD"
for pick_name in picks:
    split_name = pick_name.split("__")
    base_name = split_name[0]
    group_name = split_name[-1].replace('gr','')
    gsplit = group_name.split("-");
    gsplit[0]=str(len(gsplit[0]))
    gsplit[1]=str(len(gsplit[1]))
    topology=gsplit[0]+gsplit[1]
    #remove gr
    struct_name=pick_name[:-9]
    struct_name=truncator.find_input(struct_name, pdb_files)

    print (base_name, group_name, struct_name, gr_map[group_name])
    sn = 0;


    for chain_connections in gr_map[group_name]:
        for loopLengthRange in '4 5'.split():
            for ar1 in range(-3,0):
                for ar2 in range(-3,0):
                    n+=1; sn+=1;
                    #print(close_type, loop_length, start,end)
                    res.append( client.submit(
                        truncator.reloop, struct_name, "out/R3/11_relooped/"+base_name,
                              script_name='truncator/xml/15_redesign_loops_tl_gen_profile_SymAnnVARS.xml',
                              rosetta_bin="/home/brunette/src/Rosetta_dev/main/source/bin/rosetta_scripts.hdf5.linuxgccrelease",
                              structure_store="/home/brunette/DBs/hdf5/ss_grouped_vall_helix_shortLoop.h",
                              chain_connections=chain_connections,
                              loopLengthRange="2,"+loopLengthRange,
                              resAdjustmentRangeSide1=f"{ar1},0",
                              resAdjustmentRangeSide2=f"{ar2},0",
                              RMSthreshold=0.5,
                              test_run=False, verbose=True, add_suffix=True, skip_existing=False)
                        )


print(n)

In [11]:
n = 0
#res = []
#pick_name = "ZCON_37__numH4.5__from-21.21__to22.24__grAB-CD"
for pick_name in picks:
    split_name = pick_name.split("__")
    base_name = split_name[0]
    group_name = split_name[-1].replace('gr','')
    gsplit = group_name.split("-");
    gsplit[0]=str(len(gsplit[0]))
    gsplit[1]=str(len(gsplit[1]))
    topology=gsplit[0]+gsplit[1]
    #remove gr
    struct_name=pick_name[:-9]
    struct_name=truncator.find_input(struct_name, pdb_files)

    print (base_name, group_name, struct_name, gr_map[group_name])
    sn = 0;


    for chain_connections in gr_map[group_name]:
        for loopLengthRange in '2 3 4 5'.split():
            for ar1 in range(-3,2):
                for ar2 in range(-3,2):
                    n+=1; sn+=1;
                    #print(close_type, loop_length, start,end)
                    res.append( client.submit(
                        truncator.reloop, struct_name, "out/R3/12_relooped/"+base_name,
                              script_name='truncator/xml/15_redesign_loops_tl_gen_profile_SymAnnVARS.xml',
                              rosetta_bin="/home/brunette/src/Rosetta_dev/main/source/bin/rosetta_scripts.hdf5.linuxgccrelease",
                              structure_store="/home/brunette/DBs/hdf5/ss_grouped_vall_helix_shortLoop.h",
                              chain_connections=chain_connections,
                              loopLengthRange=f"{loopLengthRange},{loopLengthRange}",
                              resAdjustmentRangeSide1=f"{ar1},{ar1}",
                              resAdjustmentRangeSide2=f"{ar2},{ar2}",
                              RMSthreshold=0.5,
                              test_run=False, verbose=True, add_suffix=True, skip_existing=False)
                        )


print(n)

ZCON_37 AB-CD out/R3/01_cut/4.5H/ZCON_37__numH4.5__from-21.21__to22.24.pdb ['[A+B,C+D]', '[B+A,C+D]', '[A+B,D+C]', '[B+A,D+C]', '[C+D,A+B]', '[C+D,B+A]', '[D+C,A+B]', '[D+C,B+A]']
ZCON_37 A-BCD out/R3/01_cut/4.5H/ZCON_37__numH4.5__from-21.21__to22.24.pdb ['[A,B+C+D]', '[A,D+C+B]']
ZCON_15_re_loop AD-BC out/R3/01_cut/4.5H/ZCON_15_re_loop__numH4.5__from-6.83__to32.93.pdb ['[A+D,C+B]', '[D+A,C+B]', '[A+D,B+C]', '[D+A,B+C]', '[C+B,A+D]', '[C+B,D+A]', '[B+C,A+D]', '[B+C,D+A]']
ZCON_15_re_loop C-ABD out/R3/01_cut/4.5H/ZCON_15_re_loop__numH4.5__from-6.83__to32.93.pdb ['[C,B+A+D]', '[C,D+A+B]']
ZCON_127 AD-BC out/R3/01_cut/4.5H/ZCON_127__numH4.5__from-20.77__to21.47.pdb ['[A+D,C+B]', '[D+A,C+B]', '[A+D,B+C]', '[D+A,B+C]', '[C+B,A+D]', '[C+B,D+A]', '[B+C,A+D]', '[B+C,D+A]']
ZCON_127 C-ABD out/R3/01_cut/4.5H/ZCON_127__numH4.5__from-20.77__to21.47.pdb ['[C,B+A+D]', '[C,D+A+B]']
ZCON_13 AD-BC out/R3/01_cut/4.5H/ZCON_13__numH4.5__from-17.52__to22.91.pdb ['[A+D,C+B]', '[D+A,C+B]', '[A+D,B+C]', '[D+A

In [12]:
distributed.wait(res, timeout=5);

TimeoutError: Timeout

In [7]:
pick_name = "ZCON_127__numH4.5__from-20.77__to21.47__grAD-BC"
split_name = pick_name.split("__")
base_name = split_name[0]
group_name = split_name[-1].replace('gr','')
gsplit = group_name.split("-");
gsplit[0]=str(len(gsplit[0]))
gsplit[1]=str(len(gsplit[1]))
topology=gsplit[0]+gsplit[1]
#remove gr
struct_name=pick_name[:-9]
struct_name=truncator.find_input(struct_name, pdb_files)

print (base_name, group_name, struct_name, gr_map[group_name])

ZCON_127 AD-BC out/R3/01_cut/4.5H/ZCON_127__numH4.5__from-20.77__to21.47.pdb ['[A+D,C+B]', '[D+A,C+B]', '[A+D,B+C]', '[D+A,B+C]', '[C+B,A+D]', '[C+B,D+A]', '[B+C,A+D]', '[B+C,D+A]']


In [10]:
truncator.reloop( struct_name, "out/R3/11_relooped/"+base_name,
  script_name='truncator/xml/15_redesign_loops_tl_gen_profile_SymAnnVARS.xml',
  rosetta_bin="/home/brunette/src/Rosetta_dev/main/source/bin/rosetta_scripts.hdf5.linuxgccrelease",
  structure_store="/home/brunette/DBs/hdf5/ss_grouped_vall_helix_shortLoop.h",
  chain_connections="[C+B,D+A]",
  loopLengthRange="2,5",
  resAdjustmentRangeSide1=f"-1,0",
  resAdjustmentRangeSide2=f"-2,0",
  RMSthreshold=0.5,
  test_run=False, verbose=True, add_suffix=True)

cd out/R3/11_relooped/ZCON_127/ZCON_127__numH4.5__from-20.77__to21.47__ccCB-DA__llr2,5__ar1-1,0__ar2-2,0__rms0.5; /home/brunette/src/Rosetta_dev/main/source/bin/rosetta_scripts.hdf5.linuxgccrelease  \
-parser:protocol /home/ajasja/projects/truncator/truncator/xml/15_redesign_loops_tl_gen_profile_SymAnnVARS.xml \
-s /home/ajasja/projects/truncator/out/R3/01_cut/4.5H/ZCON_127__numH4.5__from-20.77__to21.47.pdb \
-indexed_structure_store:fragment_store  /home/brunette/DBs/hdf5/ss_grouped_vall_helix_shortLoop.h \
-out:suffix __ccCB-DA__llr2,5__ar1-1,0__ar2-2,0__rms0.5 \
-out:file:scorefile  ZCON_127__numH4.5__from-20.77__to21.47__ccCB-DA__llr2,5__ar1-1,0__ar2-2,0__rms0.5.sc \
-beta \
-in:file:fullatom \
-renumber_pdb 1 \
-out:file:pdb_comments true \
-run:preserve_header true \
-out:file:scorefile_format json \
-out:pdb \
-parser:script_vars chain_connections="[C+B,D+A]" loopLengthRange="2,5" resAdjustmentRangeSide1="-1,0" resAdjustmentRangeSide2="-2,0" allowed_loop_abegos="AGBA,ABBA,AGBBA,

ScriptRunResult(log_file='ZCON_127__numH4.5__from-20.77__to21.47__ccCB-DA__llr2,5__ar1-1,0__ar2-2,0__rms0.5.log', score_file='ZCON_127__numH4.5__from-20.77__to21.47__ccCB-DA__llr2,5__ar1-1,0__ar2-2,0__rms0.5.sc', pdb_file=None, status=0, run_command='cd out/R3/11_relooped/ZCON_127/ZCON_127__numH4.5__from-20.77__to21.47__ccCB-DA__llr2,5__ar1-1,0__ar2-2,0__rms0.5; /home/brunette/src/Rosetta_dev/main/source/bin/rosetta_scripts.hdf5.linuxgccrelease  -parser:protocol /home/ajasja/projects/truncator/truncator/xml/15_redesign_loops_tl_gen_profile_SymAnnVARS.xml -s /home/ajasja/projects/truncator/out/R3/01_cut/4.5H/ZCON_127__numH4.5__from-20.77__to21.47.pdb -indexed_structure_store:fragment_store  /home/brunette/DBs/hdf5/ss_grouped_vall_helix_shortLoop.h -out:suffix __ccCB-DA__llr2,5__ar1-1,0__ar2-2,0__rms0.5 -out:file:scorefile  ZCON_127__numH4.5__from-20.77__to21.47__ccCB-DA__llr2,5__ar1-1,0__ar2-2,0__rms0.5.sc -beta -in:file:fullatom -renumber_pdb 1 -out:file:pdb_comments true -run:preserve

In [43]:
res[0]

{'struct_name': 'out/R3/01_cut/4.5H/ZCON_37__numH4.5__from-21.21__to22.24.pdb',
 'dir_name': 'out/R3/09_relooped/ZCON_37',
 'chain_connections': '[A+B,C+D]',
 'loopLengthRange': '2,4',
 'resAdjustmentRangeSide1': '-3,0',
 'resAdjustmentRangeSide2': '-3,0'}

In [15]:
client.cancel(res)

In [46]:
res[0].result()

AttributeError: 'dict' object has no attribute 'result'

In [44]:
out = [r.result() for r in res]

AttributeError: 'dict' object has no attribute 'result'